# Локализация в PSLG методом полос

   Перед нами стоит одна из классических задач вычислительной геометрии: необходимо по заданному плоскому графу $G$ и точке запроса $q$ определить какому фейсу $F$ из $G$ она принадлежит.

Заметим, что если определить какое ребро выше, а какое ниже точки запроса, то можно однозначно определить какому фейсу принадлежит точка.

Для решения воспользуемся следующей идей: сначала локализуемся по координате $X$, а затем по координате $Y$.
Для этого через каждую вершину PSLG проведем вертикальную прямую. В итоге мы разбили всю плоскость на полосы.

<center><img src="images/SLABS.png"/><center>
<center><font size=5>Рис. 1. Пример разбиения плоскости на полосы </font></center>

   Теперь отсортируем полосы по координате $X$. 
Это позволит нам с помощью двоичного поиска за $O(\log{n})$ найти полосу, в которую попала точка запроса.
Заметим, что по построению ребра могут пересекаться только на границах полос.
Следовательно, внутри одной полосы ребра вертикально упорядочены.
Значит, мы можем за $O(\log{n})$ локализоваться уже в самой полосе.

## Построение полос

Рассмотрим две соседние полосы.
Заметим, что полоса справа отличается от полосы слева тем, что в ней появились и исчезли какие-то отрезки.
Значит, следующую полосу можно получить, если добавить в неё новые отрезки и удалить исчезнувшие.
Следовательно, для каждого отрезка у нас есть события двух типов.

Событию вставки будет соответствовать начало отрезка, а конец удалению.
Алгоритм становится следующим: отсортируем все события, затем будем по очереди доставать события и выполнять их.

Заметим, что вместо того, чтобы сортировать события можно завести приоритетную очередь и положить их все туда, а потом их оттуда доставать.

   Отлично! Мы получили работающий алгоритм, однако несложно придумать случай, когда он будет иметь вычислительную сложность равную $O(n^2)$ и использовать столько же памяти. 

   Например, рассмотрим отрезки вида $S_i = a_i b_i$, где $a_i = (0, i)$ и $b_i = (i, i)$ для $i = 1 \dots n$. (см. рисунок $2$)

<center><img src="images/worst_case.png"/></center>
<center><font size=5>Рис. 2. Иллюстрация </font></center>

## Персистентные деревья

   Для того чтобы улучшить использование памяти нам понадобятся персистентные деревья поиска.
   
   Давайте рассмотрим координату $X$ как время.
Двигаясь вправо по $X$, мы движемся во времени.
Пусть у нас есть изначально пустое дерево поиска.
Когда мы встречаем начало отрезка, мы добавляем его в дерево с текущей координатой $Y$.
Когда мы встречаем конец отрезка, удаляем его из дерева.
Если отрезки лежат на вертикальной прямой, события сортируются по $Y$, а если один отрезок заканчивается, а другой начинается в одной и той же точке, событие начала идет раньше.

   Вернемся к первоначальной задаче.
Применим к полосам метафору времени: одна полоса – это отрезок времени, когда ничего не происходило.
То есть, на одну полосу приходится одна версия персистентного дерева (а не отдельное дерево поиска, как раньше).
С точки зрения операции поиска ничего не изменилось. Теперь мы просто ищем по координате $X$ корень нужной версии дерева поиска, а дальше наш алгоритм повторяется.

   А вот потребление памяти уменьшилось до $O(n\log{n})$.
В персистентном дереве при операции добавления (удаления) все узлы на пути от корня до вставленной (удаленной) вершины копируются, а на перебалансировку уходит ещё $O(1)$ памяти.
Таким образом, при операциях вставки (удаления) в персистентном дереве прибавляется $O(\log{n})$ памяти.
Так как отрезков $O(n)$, то всего таких операций будет тоже $O(n)$.

   Так же очевидно, что вычислительная сложность алгоритма стала равна $O(n\log{n})$, потому что в персистентном дереве все операции выполняются за $O(\log{n})$.

## Частично персистентные деревья

   Оказывается, что использование памяти можно свести к $O(n)$.
Используем тот факт, что нам не нужна полная персистентность, то есть возможность менять и читать все версии.
Нам достаточно только частичной, то есть возможность менять и получать новые версии только из последней, но делать запросы можно по всем. Чтобы понять идею, попробуем сначала сделать что-нибудь попроще, например, частично персистентный список.

   Давайте в узле списка хранить не один указатель на следующий элемент, а два – `next` и `next2`. Дополнительно мы будем хранить номер первой версии списка, начиная с которой используется указатель `next2`. Также мы будем поддерживать таблицу (хэш-таблицу или массив), с помощью которого по версии будем получать указатель на начало списка.

### Пример
   
![](images/persistent_list00.png)
<center><font size=5>Рис. 3. Исходный список </font></center>

![](images/persistent_list01.png)
<center><font size=5>Рис. 4. В список вставлен новый узел после узла с номером $3$ </font></center>

![](images/persistent_list02.png)
<center><font size=5>Рис. 5. Опять вставили после узла с номером $3$ новый узел. При выполнении этой операции была создана копия вершины с номером $3$, обозначим её как $3'$ </font></center>

   Такую же тактику применим в деревьях.
Добавим в каждый узел дерева по дополнительному указателю `next`, номер версии и флажок, который будет показывать влево или вправо смотрит `next`.
Балансировочную информацию (размер поддерева или цвет вершины) в вершине мы будем перезаписывать, потому что эта информация может быть актуальна только для последней версии, так как все предыдущие версии дерева сбалансированы.

### Утверждение
> Вышеописанное частично персистентное дерево использует $O(n)$ памяти<br>